Loading subject data

In [1]:
from subject.braintreebank import BrainTreebankSubject

subject_id, trial_id = 1, 1
subject = BrainTreebankSubject(subject_id, cache=False)
subject.load_neural_data(trial_id)

electrode_labels = subject.get_electrode_labels()
print(f"Subject {subject.subject_identifier} has {len(electrode_labels)} electrodes at sampling rate {subject.get_sampling_rate()} Hz")
print(f"Electrode labels: {electrode_labels}\n")

example_all_electrode_data = subject.get_all_electrode_data(trial_id, window_from=0, window_to=4096)
print(f"Example all electrode data shape: {example_all_electrode_data.shape}")
print(f"Example all electrode data: {example_all_electrode_data}\n")

Subject btbank1 has 130 electrodes at sampling rate 2048 Hz
Electrode labels: ['F3aOFa2', 'F3aOFa3', 'F3aOFa4', 'F3aOFa7', 'F3aOFa8', 'F3aOFa9', 'F3aOFa10', 'F3aOFa11', 'F3aOFa12', 'F3aOFa13', 'F3aOFa14', 'F3aOFa15', 'F3aOFa16', 'F3bIaOFb1', 'F3bIaOFb2', 'F3bIaOFb3', 'F3bIaOFb4', 'F3bIaOFb5', 'F3bIaOFb6', 'F3bIaOFb7', 'F3bIaOFb8', 'F3bIaOFb9', 'F3bIaOFb10', 'F3bIaOFb11', 'F3bIaOFb12', 'F3bIaOFb13', 'F3bIaOFb14', 'F3bIaOFb15', 'F3bIaOFb16', 'F3cId1', 'F3cId2', 'F3cId3', 'F3cId4', 'F3cId5', 'F3cId6', 'F3cId7', 'F3cId8', 'F3cId9', 'F3cId10', 'T1aIb1', 'T1aIb2', 'T1aIb3', 'T1aIb4', 'T1aIb5', 'T1aIb6', 'T1aIb7', 'T1aIb8', 'T2aA1', 'T2aA2', 'T2aA3', 'T2aA4', 'T2aA5', 'T2aA6', 'T2aA7', 'T2aA8', 'T2aA9', 'T2aA10', 'T2aA11', 'T2aA12', 'T2bHa1', 'T2bHa3', 'T2bHa4', 'T2bHa5', 'T2bHa7', 'T2bHa8', 'T2bHa9', 'T2bHa10', 'T2bHa11', 'T2bHa12', 'T2bHa13', 'T2bHa14', 'T1bIc1', 'T1bIc2', 'T1bIc3', 'T1bIc4', 'T1bIc5', 'T1bIc6', 'T1bIc7', 'T1bIc8', 'F3dIe1', 'F3dIe2', 'F3dIe3', 'F3dIe4', 'F3dIe5', 'F3dIe6',

In [ ]:
electrode_locations = {electrode_label: subject.get_electrode_metadata(electrode_label)['DesikanKilliany'] for electrode_label in subject.get_electrode_labels()}
amygdala_electrode_locations = {label: location for label, location in electrode_locations.items() if 'amygdala' in location.lower()}
amygdala_electrode_locations

{'T2aA1': 'Right-Amygdala',
 'T2aA2': 'Right-Amygdala',
 'T2aA3': 'Right-Amygdala',
 'T2aA5': 'Right-Amygdala'}

In [3]:
Get the neural data index for a given subject, trial, and movie time

{'T2aA1': 'Right-Amygdala',
 'T2aA2': 'Right-Amygdala',
 'T2aA3': 'Right-Amygdala',
 'T2aA5': 'Right-Amygdala'}

In [4]:
# Get the neural data index for a given subject, trial, and movie time

from evaluation.neuroprobe.config import ROOT_DIR, SAMPLING_RATE
import os
import pandas as pd
import numpy as np

def obtain_neural_data_index(sub_id, trial_id, movie_times):
    # Data frames column IDs
    start_col, end_col = 'start', 'end'
    trig_time_col, trig_idx_col, est_idx_col, est_end_idx_col = 'movie_time', 'index', 'est_idx', 'est_end_idx'

    # Path to trigger times csv file
    trigger_times_file = os.path.join(ROOT_DIR, f'subject_timings/sub_{sub_id}_trial{trial_id:03}_timings.csv')

    trigs_df = pd.read_csv(trigger_times_file)
    display(trigs_df.head())
    print(trigs_df['type'].unique())

    last_t = trigs_df[trig_time_col].iloc[-1]
    assert np.all(movie_times < last_t), "Movie times must be less than the last trigger time"
    
    # Vectorized nearest trigger finding
    start_indices = np.searchsorted(trigs_df[trig_time_col].values, movie_times)
    start_indices = np.maximum(start_indices, 0) # handle the edge case where movie starts right at the word
    
    # Vectorized sample index calculation
    return np.round(
        trigs_df.loc[start_indices, trig_idx_col].values + 
        (movie_times - trigs_df.loc[start_indices, trig_time_col].values) * SAMPLING_RATE
    ).astype(int)

# subject 10, trial 0, movie "Cars 2"
print("For subject 10, trial 0, movie 'Cars 2'")
print(obtain_neural_data_index(10, 0, np.array([0, 1000])))

# subject 7, trial 0, movie "Cars 2"
print("For subject 7, trial 0, movie 'Cars 2'")
print(obtain_neural_data_index(7, 0, np.array([0, 1000])))

For subject 10, trial 0, movie 'Cars 2'


,type,movie_time,start_time,end_time,trig_type,index,diff
0,beginning,0.000000,211276.153251,211276.397999,manual,148150,622
1,trigger,0.083417,211276.456513,211276.473401,high_real,148772,339
2,trigger,0.208542,211276.621184,211276.638270,high_real,149111,174
3,trigger,0.291959,211276.706721,211276.723284,high_real,149285,272
4,trigger,0.417084,211276.839682,211276.855888,high_real,149557,137


['beginning' 'trigger' 'pause' 'unpause' 'end']
[ 148150 2335453]
For subject 7, trial 0, movie 'Cars 2'


,type,movie_time,start_time,end_time,trig_type,index,diff
0,beginning,0.000000,162108.308298,162108.554011,manual,28933,623
1,trigger,0.083417,162108.611594,162108.629048,high_real,29556,478
2,trigger,0.208542,162108.845048,162108.861856,high_real,30034,168
3,trigger,0.291959,162108.926974,162108.943775,high_real,30202,275
4,trigger,0.417084,162109.061252,162109.076978,high_real,30477,172


['beginning' 'trigger' 'pause' 'unpause' 'end']
[  28933 2136814]


In [5]:
def obtain_movie_start_end(sub_id, trial_id):
    # Data frames column IDs
    start_col, end_col = 'start', 'end'
    trig_time_col, trig_idx_col, est_idx_col, est_end_idx_col = 'movie_time', 'index', 'est_idx', 'est_end_idx'

    # Path to trigger times csv file
    trigger_times_file = os.path.join(ROOT_DIR, f'subject_timings/sub_{sub_id}_trial{trial_id:03}_timings.csv')

    trigs_df = pd.read_csv(trigger_times_file)
    display(trigs_df.head())

    start_row = trigs_df[trigs_df['type'] == 'beginning']
    end_row = trigs_df[trigs_df['type'] == 'end']

    display(trigs_df[trigs_df['type'].isin(['end', 'beginning', 'pause', 'unpause'])])

    start_row_movie_time = start_row[trig_time_col].iloc[-1]
    end_row_movie_time = end_row[trig_time_col].iloc[-1]

    return start_row_movie_time, end_row_movie_time

obtain_movie_start_end(6, 0), 

,type,movie_time,start_time,end_time,trig_type,index,diff
0,beginning,0.000000,799931.832360,799932.077568,manual,750390,594
1,trigger,0.083417,799932.121551,799932.138288,high_real,750984,514
2,trigger,0.208542,799932.372601,799932.389387,high_real,751498,170
3,trigger,0.291959,799932.456058,799932.472823,high_real,751668,241
4,trigger,0.417084,799932.574002,799932.590690,high_real,751909,167


,type,movie_time,start_time,end_time,trig_type,index,diff
0,beginning,0.000000,799931.832360,799932.077568,manual,750390,594
84,pause,8.259646,799940.479197,799940.670997,high_real,768059,4114
85,end,8.299967,799942.488024,799942.757538,est,772173,23405
86,beginning,8.299967,799953.897067,799954.141700,manual,795578,1899
14922,pause,1493.119612,801439.495334,801439.687382,est,3838230,111439
14923,unpause,1493.119612,801493.906661,801494.124263,manual,3949669,572
24492,pause,2450.719612,802451.785065,802451.978110,est,5911498,80659
24493,end,2450.742409,802491.169149,802491.438261,est,5992157,-1


((8.299966633, 2450.742409075),)

1.3497222222222223

In [ ]:
import os
import pandas as pd
import numpy as np
from evaluation.neuroprobe.config import ROOT_DIR, SAMPLING_RATE


def obtain_neural_data_index(sub_id, trial_id, movie_times):
    # Data frames column IDs
    start_col, end_col = 'start', 'end'
    trig_time_col, trig_idx_col, est_idx_col, est_end_idx_col = 'movie_time', 'index', 'est_idx', 'est_end_idx'

    # Path to trigger times csv file
    trigger_times_file = os.path.join(ROOT_DIR, f'subject_timings/sub_{sub_id}_trial{trial_id:03}_timings.csv')

    trigs_df = pd.read_csv(trigger_times_file)
    #display(trigs_df.head())

    last_t = trigs_df[trig_time_col].iloc[-1]
    assert np.all(movie_times < last_t), "Movie times must be less than the last trigger time"
    
    # Vectorized nearest trigger finding
    start_indices = np.searchsorted(trigs_df[trig_time_col].values, movie_times)
    start_indices = np.maximum(start_indices, 0) # handle the edge case where movie starts right at the word
    
    # Vectorized sample index calculation
    return np.round(
        trigs_df.loc[start_indices, trig_idx_col].values + 
        (movie_times - trigs_df.loc[start_indices, trig_time_col].values) * SAMPLING_RATE
    ).astype(int)

# subject 10, trial 0, movie "Cars 2"
print("For subject 10, trial 0, movie 'Cars 2'")
print(obtain_neural_data_index(10, 0, np.array([0, 1000])))

# subject 7, trial 0, movie "Cars 2"
print("For subject 7, trial 0, movie 'Cars 2'")
print(obtain_neural_data_index(7, 0, np.array([0, 1000])))

For subject 10, trial 0, movie 'Cars 2'
[ 148150 2335453]
For subject 7, trial 0, movie 'Cars 2'
[  28933 2136814]
